In [33]:
# Run this cell only once! 
from pyspark import SparkConf, SparkContext
conf = SparkConf() \
  .setAppName("MovieLensALS") \
  .set("spark.executor.memory", "2g")
spark_context = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-b698fc44b8a5>:3 

In [2]:
# For testing only.

lines = spark_context.textFile("../README.md")  
words = lines.flatMap(lambda line: line.split())  
count = words.count()  
print("Word Count: " + str(count)) 

Word Count: 14


In [3]:
# Try recommender system
import pandas as pd
import numpy as np
import scipy.sparse as sp

# This is actually not necessary.
def preprocess_data(data):
    """preprocessing the text data, conversion to numerical array format."""
    def deal_line(line):
        pos = line[0]
        rating = line[1]
        row, col = pos.split("_")
        row = row.replace("r", "")
        col = col.replace("c", "")
        return int(row), int(col), float(rating)
    
    def statistics(data):
        min_row = np.min(data[:,0])
        max_row = np.max(data[:,0])
        min_col = np.min(data[:,1]) 
        max_col = np.max(data[:,1])
        return min_row, max_row, min_col, max_col

    # parse each line
    data_matrix = np.apply_along_axis(deal_line,axis=1,arr=data)

    min_row, max_row, min_col, max_col = statistics(data_matrix)
    ratings = sp.lil_matrix((int(max_row), int(max_col)))
    for row, col, rating in data_matrix:
        ratings[row - 1, col - 1] = rating
    return ratings, data_matrix

#data = pd.read_csv("../data/data_train.csv")
#__, data_new = preprocess_data(data.as_matrix().reshape((-1,2)))
#train_pd = pd.DataFrame({'user':data_new[:,0],'item':data_new[:,1],'rating':data_new[:,2]})

In [40]:
from pyspark.sql import SQLContext
from helpers import load_data
import scipy.sparse as sp
print("load")
train_raw = load_data("../data/data_train.csv")
print("find")
rows, cols, ratings = sp.find(train_raw) 
print("sql_context")
context = SQLContext(spark_context)
#df_panda = pd.DataFrame([(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],["user", "item", "rating"])
#df_panda = pd.DataFrame([(0, 0, 1, 1, 2, 2),(0,1,1,2,1,2),(4.0,2.0,3.0,4.0,1.0,5.0)],["user", "item", "rating"])
print("panda dataframe")
train_pd = pd.DataFrame({'user':rows+1,'item':cols+1,'rating':ratings})
print("sql dataframe")
train = context.createDataFrame(train_pd)

number of items: 10000, number of users: 1000
find
sql_context
panda dataframe
sql dataframe


In [41]:
train.show()

+----+------+----+
|item|rating|user|
+----+------+----+
|   1|   4.0|  44|
|   1|   3.0|  61|
|   1|   4.0|  67|
|   1|   3.0|  72|
|   1|   5.0|  86|
|   1|   4.0|  90|
|   1|   3.0| 108|
|   1|   3.0| 114|
|   1|   2.0| 120|
|   1|   5.0| 135|
|   1|   4.0| 152|
|   1|   3.0| 165|
|   1|   3.0| 182|
|   1|   3.0| 310|
|   1|   1.0| 318|
|   1|   3.0| 333|
|   1|   2.0| 355|
|   1|   4.0| 390|
|   1|   4.0| 401|
|   1|   2.0| 410|
+----+------+----+
only showing top 20 rows



In [42]:
# Apply alternating least squares
from pyspark.ml.recommendation import ALS

# TODO: split in test and training data for cross validation.

als = ALS(rank=30, maxIter=5)
model = als.fit(train)
print('solved for rank',model.rank)
#print(model.userFactors.orderBy("id").collect())
#print(model.itemFactors.orderBy("id").collect())

solved for rank 30


In [36]:
from helpers import load_data
ratings = load_data('../data/sampleSubmission.csv')
rows, cols, __ = sp.find(ratings)
rows += 1
cols += 1
test_pd = pd.DataFrame({'user':rows,'item':cols})
test = context.createDataFrame(test_pd)
#test = context.createDataFrame([(1, 2), (1, 1), (2, 1)], ["user", "item"])
test.show()

number of items: 10000, number of users: 1000
+----+----+
|item|user|
+----+----+
|   1|  37|
|   1|  73|
|   1| 156|
|   1| 160|
|   1| 248|
|   1| 256|
|   1| 284|
|   1| 400|
|   1| 416|
|   1| 456|
|   1| 474|
|   1| 495|
|   1| 515|
|   1| 518|
|   1| 521|
|   1| 559|
|   1| 596|
|   1| 614|
|   1| 621|
|   1| 661|
+----+----+
only showing top 20 rows



In [14]:
predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])

In [15]:
predictions[0]

Row(item=1, user=7833, prediction=3.4777028560638428)

In [16]:
predictions[1]

Row(item=1, user=3986, prediction=2.6524620056152344)

In [26]:
als.explainParams()

"alpha: alpha for implicit preference (default: 1.0)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. (default: 10)\nfinalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)\nimplicitPrefs: whether to use implicit preference (default: False)\nintermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK)\nitemCol: column name for item ids. Ids must be within the integer value range. (default: item)\nmaxIter: max number of iterations (>= 0). (default: 10, current: 5)\nnonnegative: whether to use nonnegative constraint for least squares (default: False)\nnumItemBlocks: number of item blocks (default: 10)\nnumUserBlocks: number of user blocks (default: 10)\npredictionCol: prediction column name. (default: prediction)\nrank: rank of the factorization (default: 10, current: 30)\nratingCol: column name for ratings (def

In [29]:
type(predictions[0])
print(model.userFactors.orderBy("id").collect())

pyspark.sql.types.Row

In [43]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

#train = context.createDataFrame([(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],
#                                 ["user", "item", "rating"])
#test = context.createDataFrame([(0, 0), (0, 1), (1, 1), (1, 2), (2, 1), (2, 2)], ["user", "item"])


als = ALS()
model = als.fit(train)
param_map = ParamGridBuilder() \
                    .addGrid(als.rank, [20, 21]) \
                    .addGrid(als.maxIter, [10, 15]) \
                    .addGrid(als.regParam, [1.0, 10.0]) \
                    .build()
evaluatorR = RegressionEvaluator(metricName="rmse", labelCol="rating")
cvExplicit = CrossValidator(estimator=als, estimatorParamMaps=param_map, evaluator=evaluatorR)
cvModelExplicit = cvExplicit.fit(train)
predsExplicit = cvModelExplicit.bestModel.transform(test)
predsExplicit.show()

+----+----+----------+
|item|user|prediction|
+----+----+----------+
| 148| 148|  3.215077|
| 148| 463|  3.377499|
| 148| 496| 3.6420448|
| 148|3175|  3.296978|
| 148|3794| 2.8517187|
| 148|3918|  3.342958|
| 148|4519| 3.4460375|
| 148|5300|  3.169028|
| 148|5803| 3.0466917|
| 148|6654| 3.2227583|
| 148|6658| 3.1316009|
| 148|7240|  3.047616|
| 148|7833| 3.6500893|
| 148|7880| 3.5918689|
| 148|7993| 3.6432185|
| 148|8592| 3.5379062|
| 148|9376| 3.4238617|
| 148|9465| 3.4224896|
| 148| 623| 3.2936347|
| 148| 737| 3.4555721|
+----+----+----------+
only showing top 20 rows



In [52]:
print('best model: \n rank:',cvModelExplicit.bestModel.rank)
print('maxIter:',cvModelExplicit.bestModel.getParam('maxIter'))
print('regParam: \n',cvModelExplicit.bestModel.regParam)

best model: 
 rank: 21


AttributeError: 'ALSModel' object has no attribute 'maxIter'